In [1]:
# Import Dependencies
import pandas as pd
import hvplot.pandas
import requests
import json
from holoviews import opts
import holoviews as hv
hv.extension('bokeh')

# Import the API key
from api_keys import geoapify_key

In [2]:
# Import data from csv file
top_30_df = pd.read_csv('Resources/top_30_df.csv')

top_30_df.head()

,Ticker,company,address,city,state,zip,country,address_combined,Average Manager Age
0,PLTR,Palantir Technologies Inc.,1200 17th Street,Denver,CO,80202,United States,"1200 17th Street, Denver, CO, 80202, United St...",46
1,TLRY,"Tilray Brands, Inc.",265 Talbot Street West,Leamington,ON,N8H 5L4,Canada,"265 Talbot Street West, Leamington, ON, N8H 5L...",55
2,TSLA,"Tesla, Inc.",1 Tesla Road,Austin,TX,78725,United States,"1 Tesla Road, Austin, TX, 78725, United States",45
3,LCID,"Lucid Group, Inc.",7373 Gateway Boulevard,Newark,CA,94560,United States,"7373 Gateway Boulevard, Newark, CA, 94560, Uni...",53
4,AAPL,Apple Inc.,One Apple Park Way,Cupertino,CA,95014,United States,"One Apple Park Way, Cupertino, CA, 95014, Unit...",58


In [3]:
# Initialize the columns for the dataframe
top_30_df["Lat"] = ""
top_30_df["Lng"] = ""

top_30_df.head()

,Ticker,company,address,city,state,zip,country,address_combined,Average Manager Age,Lat,Lng
0,PLTR,Palantir Technologies Inc.,1200 17th Street,Denver,CO,80202,United States,"1200 17th Street, Denver, CO, 80202, United St...",46,,
1,TLRY,"Tilray Brands, Inc.",265 Talbot Street West,Leamington,ON,N8H 5L4,Canada,"265 Talbot Street West, Leamington, ON, N8H 5L...",55,,
2,TSLA,"Tesla, Inc.",1 Tesla Road,Austin,TX,78725,United States,"1 Tesla Road, Austin, TX, 78725, United States",45,,
3,LCID,"Lucid Group, Inc.",7373 Gateway Boulevard,Newark,CA,94560,United States,"7373 Gateway Boulevard, Newark, CA, 94560, Uni...",53,,
4,AAPL,Apple Inc.,One Apple Park Way,Cupertino,CA,95014,United States,"One Apple Park Way, Cupertino, CA, 95014, Unit...",58,,


In [4]:
# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v1/geocode/search"

In [5]:
for index, row in top_30_df.iterrows():
    params = {
        "text": row["address_combined"],
        "apiKey": geoapify_key,
        format: "json"
    }

    # Run request
    response = requests.get(base_url, params=params).json()

    # Extract lat/lng
    try:
        top_30_df.loc[index, "Lat"] = response["features"][0]["properties"]["lat"]
        top_30_df.loc[index, "Lng"] = response["features"][0]["properties"]["lon"]
    except:
        print(f"Cannot find lat/lng for {row['address_combined']}")

In [12]:
top_30_df[["Ticker", "company", "state", "address_combined", "Lat", "Lng"]].to_csv("Resources/geo_info.csv", index=False)

In [11]:
# Grouping by state and counting companies in each state
state_concentration = top_30_df.groupby('state')['Ticker'].count().reset_index()

# Renaming columns for clarity
state_concentration.columns = ['State', 'Company Count']

# Sorting in descending order to find the most concentrated state
most_concentrated_state = state_concentration.sort_values(by='Company Count', ascending=False).iloc[0]


In [13]:
state_concentration

,State,Company Count
0,AL,1
1,AZ,1
2,CA,7
3,CO,2
4,FL,1
5,KS,1
6,MI,1
7,NC,1
8,NJ,1
9,NY,3


In [14]:
most_concentrated_state

State            CA
Company Count     7
Name: 2, dtype: object

In [7]:
# Create the map plot using hvplot
map_plot = top_30_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    hover_cols=["Ticker", "company", "state"],
    alpha=0.3,
    height=500,
    width=500,
    frame_width=500,
    frame_height=500,
    tiles="OSM"
)

# Set the initial view to center around the United States
map_plot = map_plot.redim.range(
    Lng=(-130, -60),  # Longitude range for the United States
    Lat=(24, 50)  # Latitude range for the United States
)

map_plot.opts(
    opts.Points(size=10, color='blue', tools=['hover']),
    opts.WMTS(alpha=0.5),
)

hvplot.save(map_plot, 'Output/map_plot.png')